# Prediction

In [396]:
input_cardinality=20
output_cardinality=7
#change output_cardinality =[1,2,3,4,5,6,7,8,9,10,15,20]

In [1]:
import torch
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import sklearn.model_selection as sk
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import joblib

In [398]:
#import pytorch_lightning as pl
import time
import copy
from tqdm import tqdm


In [399]:
torch.manual_seed(0)
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [400]:
#from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
#from pytorch_lightning.loggers import TensorBoardLogger

In [401]:
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")

In [402]:
#pl.seed_everything(43)

In [403]:
class DATA:
    def __init__(self, data_path):
        self.data_path = data_path
        self.cols = 'ThetaXHG'
    def get_data(self):
        return self.read_data()
    
    def read_data(self): #polymorphism
        return pd.read_csv(self.data_path)

In [404]:
class THETADATASET(Dataset):
    #convert to pytorch dataset
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        sequence, output = self.X[idx], self.y[idx]
        return (torch.from_numpy(sequence.reshape(-1)), torch.from_numpy(output.reshape(-1)))
        #return dict(sequence=torch.tensor(sequence.reshape(-1),dtype=torch.float64), label=torch.tensor(output.reshape(-1),dtype=torch.float64))

In [405]:
DATA_PATH = "../DATASET/REALTIME_DATA/alldata.csv"
data = DATA(DATA_PATH)
samples = data.get_data()

In [406]:
samples

,HG_J0,HG_J1,HG_J2,HG_J3,HG_J4,HG_J5,HG_J6,HG_J7,HG_J8,HG_J9,...,RH_J9,RH_J10,RH_J11,RH_J12,RH_J13,RH_J14,RH_J15,ThetaXHG,ThetaYHG,ThetaZHG
0,0.545,0.724681,0.567105,0.416667,0.624017,0.608333,0.506329,0.308333,0.378247,1,...,-0.195763,0.693640,0,1.466225,0.223258,-0.292838,1.687952,-0.001207,0,0
1,0.545,0.724681,0.567105,0.421667,0.624891,0.615000,0.508075,0.315000,0.380009,1,...,-0.195763,0.693640,0,1.466050,0.223212,-0.292807,1.687925,0.009104,0,0
2,0.545,0.724431,0.566667,0.420000,0.624891,0.611667,0.508075,0.311667,0.380889,1,...,-0.195763,0.693640,0,1.465492,0.223066,-0.292705,1.687838,0.005889,0,0
3,0.545,0.724181,0.566228,0.421667,0.624891,0.610000,0.508512,0.310000,0.383972,1,...,-0.195763,0.693640,0,1.465392,0.223040,-0.292687,1.687823,0.007015,0,0
4,0.545,0.724181,0.566228,0.425000,0.624891,0.613333,0.508512,0.311667,0.383972,1,...,-0.195763,0.693640,0,1.465212,0.222993,-0.292654,1.687794,0.011417,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1443,0.547,0.692923,0.369298,0.665000,0.772489,0.691667,0.721955,0.413333,0.663144,1,...,-0.217956,0.695774,0,1.435448,0.220266,-0.285573,1.676194,0.330298,0,0
1444,0.547,0.692673,0.367982,0.665000,0.772489,0.691667,0.724138,0.413333,0.665346,1,...,-0.217956,0.695774,0,1.425746,0.217689,-0.283673,1.673708,0.335613,0,0
1445,0.547,0.692923,0.367544,0.665000,0.772489,0.691667,0.724138,0.413333,0.665346,1,...,-0.217956,0.695774,0,1.418387,0.215728,-0.282196,1.671575,0.336424,0,0
1446,0.547,0.692923,0.367544,0.665000,0.772489,0.691667,0.724138,0.413333,0.665786,1,...,-0.217956,0.695774,0,1.412639,0.214192,-0.281020,1.669761,0.336424,0,0


In [407]:
samples[data.cols]

0      -0.001207
1       0.009104
2       0.005889
3       0.007015
4       0.011417
          ...   
1443    0.330298
1444    0.335613
1445    0.336424
1446    0.336424
1447    0.336741
Name: ThetaXHG, Length: 1448, dtype: float64

In [408]:
class LSTMNET(nn.Module):
    def __init__(self, batch_size, input_len, output_len, lstm_units = 4, num_layers=1):
        super(LSTMNET, self).__init__()
        self.input_len = input_len
        self.output_len = output_len
        self.num_layers = num_layers
        self.batch_size = batch_size
        #input_size = no of features = 1
        #hidden_size = no of lstm units in the layer
        #num_layers = no of lstm layers
        self.lstm_units = lstm_units
        self.lstm1 = nn.LSTM(input_size= 1, hidden_size= lstm_units, num_layers=num_layers,batch_first=True, dropout=0.6)
        
        self.linear0 = nn.Linear(in_features= 20, out_features=10)
        
        self.linear1 = nn.Linear(in_features= lstm_units, out_features=10)
        self.linear2 = nn.Linear(in_features= 10, out_features=10)
        self.ll = nn.Linear(in_features= 10, out_features=output_len)
        self.hidden = (torch.zeros(1*self.num_layers, self.batch_size, self.lstm_units).double(), torch.zeros(1*self.num_layers, self.batch_size, self.lstm_units).double())
        #print(self.hidden[0].device)
        #print(self.hidden.shape)
        #self.hidden[0]= self.hidden[0].to(device)
        #self.hidden[1] = self.hidden[1].to(device)
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
    
    def reset_hidden_states(self,bs):
        self.hidden = (torch.zeros(1*self.num_layers, bs, self.lstm_units).double(), torch.zeros(1*self.num_layers, bs, self.lstm_units).double())
        
    def forward(self,x):
        #print(x.shape)
        #print(x)
        #print(x.unsqueeze(-1).shape)
        #print(self.hidden.shape)
        #print(self.hidden)
        lstm_out, (h,c) = self.lstm1(x.unsqueeze(-1), self.hidden)
        self.hidden= (h.detach(),c.detach())
        #c.detach_()
        #h.detach_()
        #self.hidden = (h.detach(), c.detach())
        #print(ht.shape)
        #ht=ht.to(device)
        #ct=ct.to(device)
        
        lstm_out = lstm_out[:,-1,:]
        #print(ht.shape)
        #either lstm_out goes to next or ht goes
        #lstm_out= h.detach()[-1]
        lin1_out = self.linear1(lstm_out)
        #Add RELU
        #lin0_out = F.relu(self.linear0(x))
        ll_out = self.ll(lin1_out)
        #x = self.linear0(x)
        #print(x.shape)
        
        #x = self.linear2(x)
        #Add RELU
        return ll_out

In [409]:
model_ckp_path = "../MODEL_CHECKPOINTS/model_lookahead"+str(output_cardinality)+".pth"
scaler_filename = "../SCALER_DUMPS/min_max_scaler_lookahead"+str(output_cardinality)+".save"
batch_size = 1
predictor = LSTMNET(batch_size=batch_size,input_len=input_cardinality,output_len=output_cardinality)
predictor.load_state_dict(torch.load(model_ckp_path))
#net2.eval()
predictor.reset_hidden_states(bs=1)
scaler = joblib.load(scaler_filename) 

predictor.eval() 
predictor = predictor.double()

/home/toor/anaconda3/envs/py37/lib/python3.7/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.6 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [410]:
#Generate predictions
j = input_cardinality
prediction_mismatch = list()
columns_= ['ThetaXHG_PREDICTION', 'ThetaXHG_GROUND']
for i in range(samples[data.cols].shape[0]-input_cardinality+1):
    X = samples[data.cols][i:i+input_cardinality]    
    if j <=samples[data.cols].shape[0]-1:
        y_ground = samples[data.cols][i+input_cardinality]
    else:
        y_ground = ''
    X_scaled = scaler.transform(X.to_numpy().reshape(X.shape[0],1))
    X_torch_scaled = torch.tensor(X_scaled.transpose(), dtype=torch.float64)
    y_pred_scaled = predictor(X_torch_scaled)
    y_pred = scaler.inverse_transform(y_pred_scaled.detach().numpy())
    prediction_mismatch.append([y_pred[0][output_cardinality-1],y_ground])
    j +=1

In [411]:
pd.DataFrame(prediction_mismatch, columns=columns_).to_csv('../RESULTS_PREDICTION/PREDICTION_lookahead'+str(output_cardinality)+".csv", index=False)

In [11]:
x = [i for i in range(10)]
input_cardinality=20

In [12]:
x[-input_cardinality-1:-1]

[0, 1, 2, 3, 4, 5, 6, 7, 8]